In [ ]:
#recording come in edf (European Data Format) format. Use mne.io.read_raw_edf()
#information comes in xlsx format: convert to csv using xlsx2csv package - read with pandas

In [ ]:
files = ['EPIstim_001_NPS_stim_02.edf', 
         'EPIstim_001_Probeschlaf_EOG_EMG_4h.edf', 'EPIstim_001_Shamnacht.edf', 
         'EPIstim_001_StimNacht.edf', 'EPIstim_001_StimNacht_Backup.edf']
fxlsx = 'ElektrodenInfo.xlsx'
file = files[-2]

In [ ]:
path = '/mnt/raid/data/GreifswaldData/iEEG/' #/EPI_1001/EEG'
subject = 'EPI_1001/'
folder = 'EEG/'

In [ ]:
full_path = path + subject + folder

In [ ]:
#import necessary packages
#for processing 'brain-data'
import mne

#for data handling
import pandas as pd

#for functionalities
import numpy as np

In [ ]:
#read xlsx information file to collect necessary information to read the raw edf-files
#o.e. what is the eog, what the misc, what to excude, ...

df = pd.read_excel(full_path+fxlsx, sheet_name='subject01', header=0)
df

#identify eog (Electrooculography = technique for measuring the corneo-retinal standing potential 
#that exists between the front and the back of the human eye) 
eogs = #todo

#identify all channels to exclude in read (i.e. channels that are not used, empty, or not of interest)
exclude = []

In [ ]:
raw = mne.io.read_raw_edf(full_path+file, eog=[])

In [ ]:
print(raw)
print(raw.info)

In [ ]:
#do he very simple things like Power Spectral Densities or Visualisation of Channels
raw.compute_psd(method='welch', 
                fmin=0, fmax=50, #fmin, fmax: lower- and upper-bound on frequs of interest
                tmin = 2, tmax=17, #tmin, tmax: first & last times to include in seconds
                picks = [], exclude = [] #picks, exclude: channels to in-/exclude
               ) #n_jobs: number of jobs to run in parallel
raw.plot(duration=2, #duration: time window(s) to plot
         n_channels=8 #n_channels: number of channels to plot at once, defaults to 20
        ) #events=array/None: events to show with vertical bars

In [ ]:
#components that best capture artifacts can be identified using ICA
#ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
#ica.fit(raw)
#ica.exclude = [1,2]
#ica.plot_properties(raw, picks=ica.exclude)

In [ ]:
#sure what components to remove, pass them as 'exclude' param (then again apply ICA)

#excludes = [list of strings which channels to exclude]

In [ ]:
#detection of experimental events
#options are: 1/2 (auditory, left/right ear), 3/4 (visual, left/right), 
#5 (smiley face), 32 (subject button press)
events = mne.find_events(raw, stim_channel='the channel that was the stimulation recording')
#output: np.array([sample_number, ignore, integer_event_ID])

#plot the whole thing 
fig = mne.viz.plot_events(
    events, event_id=event_dict, sfreq=raw.info["sfreq"], first_samp=raw.first_samp
)

# Epoching

## Rejection Dictionary
$\texttt{reject=reject_criteria}$:
Reject epochs based on maximum peak-to-peak signal amplitude (PTP), i.e. the absolute difference between the lowest and the highest signal value. In each individual epoch, the PTP is calculated for every channel. If the PTP of any one channel exceeds the rejection threshold, the respective epoch will be dropped.

In [ ]:
#specify data quality constraints in a dict
reject_criteria = dict(
    mag=4000e-15,  # 4000 fT
    grad=4000e-13,  # 4000 fT/cm
    eeg=150e-6,  # 150 µV
    eog=250e-6,
)  # 250 µV

In [ ]:
#create epochs object
#pass event_id as event dict parameter
epochs = mne.Epochs(
    raw,
    events,
    event_id=event_dict,
    tmin=-0.2,
    tmax=0.5,
    reject=reject_criteria,
    preload=True,
) #out:

In [ ]:
#one can pool across stimuli presentations etc. 
#but more interestingly is that you can plot epochs channel-wise:
epochs.plot_image(picks=["MEG 1332", "EEG 021"])

# Time-Frequency Analysis

In [ ]:
#package mne.time_frequency includes several tools for time-frequs-computations
#among others: method to identify power of different frequs using Morlet wavelets:
frequencies = np.arange(7, 30, 3)
power = mne.time_frequency.tfr_morlet(
    epochs, n_cycles=2, return_itc=False, freqs=frequencies, decim=3
)
power.plot(['channel_name'])